# Problem Description
Wake county located in North Carolina is one of the fastest growing counties in the USA large influx of immigrants from other states and nationalities calling it their home.
The highly educated workforce with a good mix of young and middle aged population drives the economy of the state and opens up tremendous  possibilities for new businesses to thrive.North carolina has been influenced greatly by settlers from New England and Hispanic immigrants.North carolinais also famous for its barbecue and bacon but other cuisines are starting to becoming popular.
This opens up a possibility to create a recommender system for anyone who wants to open a new restaurant that can be successful in the area based on the information
1. Demographics and Income
2. Existing restaurants density in the area(To identify any untapped market)
3. Types of restaurants based on cuisines
4. Existing Restaurant quality based on reviews and food inspections
5. Farmer's Market around the various neighbourhood

Wake county has 12 Towns and  46 Communities and for this work we will obtain the following datasets from https://data-wake.opendata.arcgis.com/datasets
1. Wake county restaurants Datasets(csv)
2. Wake county food inspections datasets(csv)
3. Wake county demographic information(csv)
4. Farmers Market http://www.wakegov.com/humanservices/publichealth/information/promotion/Pages/farmersmarkets.aspx
The data from the above datasets will be read via pandas dataframe and cleaned up to classify Restaurants to Townships and and quality index score.Then we will be able to cluster the restaurants in the area and as well as correlate the quality of restaurants again based on cuisine to the neighbourhood which would potentially provide us insights on the type of restaurant that can be setup. We will be using Foursquare api to obtain restaurant reviews in the locality to do additional correlation that will strengthen the model.We will finally demonstrate the efficacy with vuisualisation plots and graphs that will aid in the final decision

## Recommendation Success Criteria
We will determine the success of the recommender system by providing potential locations it can be set up for maximum success

### Import relevant libraries


In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlt
from bs4 import BeautifulSoup
import requests
import folium

### Raleigh Demographic Distribution

In [2]:
website_url=requests.get('https://en.wikipedia.org/wiki/Raleigh,_North_Carolina').text
soup=BeautifulSoup(website_url,'lxml')
DemoTable=soup.find_all('table')[3]
dfRal_demo=pd.DataFrame(columns=['Race','Year=2010','Year=1990','Year=1970','Year=1950'],index=[1])
rows=DemoTable.find_all('tr')
l = []
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    l.append(row)
dfRal_demo=pd.DataFrame(l, columns=['Race','Year=2010','Year=1990','Year=1970','Year=1950'])

In [3]:
dfRal_demo.drop(0)

,Race,Year=2010,Year=1990,Year=1970,Year=1950
1,White,57.5%,69.2%,76.6%,72.7%
2,—Non-Hispanic whites,53.3%,68.4%,76.4%,n/a
3,Black or African American,29.3%,27.6%,22.7%,27.2%
4,Hispanic or Latino (of any race),11.4%,1.4%,0.6%,n/a
5,Asian,4.3%,2.5%,0.3%,0.1%


### Durham Demographic Distribution

In [4]:
durham_demo=pd.read_csv("C:\Python3\Coursera_Captstone\Durham_Demographic.csv")

In [5]:
durham_demo

,Year=2010,Race
0,42.45%,White
1,37.9%,Non-Hispanic White
2,40.96%,Black or African American
3,5.07%,Asian American
4,0.51%,Native American
5,0.07%,Native Hawaiian or Other Pacific Islander
6,8.28%,some other race
7,2.66%,two or more races
8,14.22%,Hispanic or Latino of any race


### Demographic Information of the Various Towns in Wakecounty

In [6]:
towns_demo=pd.read_csv("C:\Python3\Coursera_Captstone\Demography_ByTOWNS.csv")

### Get Geolocation for each of the cities and towns within Wake county

In [7]:
towns_demo

,City,Race,Percent
0,Angier,White,67.42
1,Angier,Black or African American,23.37
2,Angier,Asian American,0.94
3,Angier,Native American,0.41
4,Angier,Native Hawaiian or Other Pacific Islander,0.03
5,Angier,some other race,6.23
6,Angier,two or more races,1.61
7,Angier,Hispanic,12.7
8,Apex,White,80
9,Apex,Black or African American,8


In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from datetime import datetime
print('Libraries imported.')

Libraries imported.


In [9]:
Cities=['Durham,NC','Raleigh,NC']
c=[]
for city in Cities:
    geolocator = Nominatim(user_agent="CapstoneProject")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of '+city+' are {}, {}.'.format(latitude, longitude))   
    c.append([city,latitude,longitude])
    # create map of Toronto using latitude and longitude values
    # map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
df_cities=pd.DataFrame(c,columns=['City','Latitude','Longitude'])
Towns=['Angier','Apex','Cary','fuquay varina','Garner','Holly Springs','Knightdale','Morrisville','Rolesville','wake forest','Wendell','Zebulon']
t=[]
for town in Towns:
    geolocator = Nominatim(user_agent="CapstoneProject")
    location = geolocator.geocode(town)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of '+town+' are {}, {}.'.format(latitude, longitude))
    t.append([town,latitude,longitude])
df_towns=pd.DataFrame(t,columns=['City','Latitude','Longitude'])

The geograpical coordinate of Durham,NC are 36.0181316, -78.8751582.
The geograpical coordinate of Raleigh,NC are 35.7803977, -78.6390989.
The geograpical coordinate of Angier are 35.5071045, -78.7391813.
The geograpical coordinate of Apex are 35.7325352, -78.8505516.
The geograpical coordinate of Cary are 35.7883046, -78.7811964.
The geograpical coordinate of fuquay varina are 35.61551875, -78.8164514666885.
The geograpical coordinate of Garner are 35.7112642, -78.6141709.
The geograpical coordinate of Holly Springs are 35.6512655, -78.8336218.
The geograpical coordinate of Knightdale are 35.7878975, -78.4822938.
The geograpical coordinate of Morrisville are 35.824341, -78.8300321.
The geograpical coordinate of Rolesville are 35.9232862, -78.4573914.
The geograpical coordinate of wake forest are 35.9803138, -78.5103731.
The geograpical coordinate of Wendell are 35.780987, -78.3697213.
The geograpical coordinate of Zebulon are 35.8243208, -78.3147199.


In [10]:
TOWNS_demo=pd.read_csv("C:\Python3\Coursera_Captstone\Demography_ByTOWNS.txt")

In [11]:
df_cities

,City,Latitude,Longitude
0,"Durham,NC",36.018132,-78.875158
1,"Raleigh,NC",35.780398,-78.639099


In [12]:
dfCityRace=pd.merge(TOWNS_demo,df_towns,on=['City'],how='outer')

In [13]:
uniqueCity=dfCityRace[['City','Latitude','Longitude']]
citylatlng=uniqueCity.drop_duplicates()

## Using Foursquare API to retrieve restaurants in Raleigh /Durham and Towns 

In [14]:
wakecounty_inspections=pd.read_csv("C:\Python3\Coursera_Captstone\Food_Inspections.csv")
wcmostrecent=wakecounty_inspections
sorted=wcmostrecent.sort_index(by='DATE_')
wcmostrecent.shape

c:\users\itdxv01\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  This is separate from the ipykernel package so we can avoid doing imports until


(34539, 8)

In [15]:
result=sorted.drop_duplicates('HSISID',keep='last').values
newarr=pd.DataFrame(result,index=result[:,0])
newarr.sort_values(by=1)
newarr
newarr.columns=['OBJECTID', 'HSISID', 'SCORE', 'DATE_', 'DESCRIPTION', 'TYPE',
       'INSPECTOR', 'PERMITID']
newarr.sort_values(by='HSISID')

,OBJECTID,HSISID,SCORE,DATE_,DESCRIPTION,TYPE,INSPECTOR,PERMITID
11844,11844,4092010018,97,2019-01-30T00:00:00.000Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Brittny Thomas,1604
8434,8434,4092010031,94,2018-08-27T00:00:00.000Z,"*NOTICE* EFFECTIVE JANUARY 1, 2019, THE NC FOO...",Inspection,Maria Powell,7735
7849,7849,4092010035,96.5,2019-03-13T00:00:00.000Z,Contact phone number: 919-422-3747 *NOTICE* AS...,Inspection,Joshua Volkan,125
8432,8432,4092010037,97.5,2019-03-04T00:00:00.000Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Kaitlyn Yow,5815
8083,8083,4092010052,94,2018-11-19T00:00:00.000Z,"*NOTICE* EFFECTIVE JANUARY 1, 2019, THE NC FOO...",Inspection,Angela Jacobs,8629
9291,9291,4092010065,96.5,2019-02-04T00:00:00.000Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,6322
8102,8102,4092010069,96.5,2019-03-13T00:00:00.000Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Michael Robinson,2932
7739,7739,4092010070,98.5,2018-11-19T00:00:00.000Z,"For ""time held for quality"" sausage to be used...",Inspection,Joanne Rutkofske,6030
10535,10535,4092010072,99,2019-03-12T00:00:00.000Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Kaitlyn Yow,2267
8413,8413,4092010073,99.5,2019-04-02T00:00:00.000Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Maria Powell,5297


In [16]:
from datetime import datetime

newarr['DATE_']=pd.to_datetime(newarr['DATE_'])
newarr['DATE_']=newarr['DATE_'].dt.date
newarr['DATE_']
newarr.sort_values(by='HSISID')
newarr.to_csv('newarr.csv')
newarr.head(5)

,OBJECTID,HSISID,SCORE,DATE_,DESCRIPTION,TYPE,INSPECTOR,PERMITID
31484,31484,4092440291,98.5,2016-07-25,Inspection made with Mr. Pablo.,Inspection,Pamela Scott,3655
9401,9401,4092440058,90,2016-08-09,"Wet, dripping mops were found propped against ...",Inspection,Caroline Suggs,2889
30556,30556,4092440446,0,2016-08-12,FOOD (WCES) APPROVAL SIGN OFF ON SCHOOL BUILDI...,Inspection,Rebecca Robbins,19667
11528,11528,4092440017,88,2016-08-17,NaN,Inspection,Anne-Kathrin Bartoli,145
23291,23291,4092440351,91,2016-08-23,This school does not have a permitted school c...,Inspection,Anne-Kathrin Bartoli,19685


In [17]:
wakecounty_restaurants=pd.read_csv("C:\Python3\Coursera_Captstone\Restaurants_in_Wake_County.csv")
wakecounty_restaurants
wcrest = pd.DataFrame(wakecounty_restaurants,columns=['HSISID', 'NAME','CITY','RESTAURANTOPENDATE','FACILITYTYPE','X','Y'])
wcres=wcrest.loc[wcrest['FACILITYTYPE'].isin(['Restaurant'])]
wcres.shape
wcres.head(10)

,HSISID,NAME,CITY,RESTAURANTOPENDATE,FACILITYTYPE,X,Y
0,4092017831,DUNKIN DONUTS #350292,HOLLY SPRINGS,2018-04-25T00:00:00.000Z,Restaurant,-78.830142,35.637386
1,4092017308,WENDY'S #6319,HOLLY SPRINGS,2016-09-02T00:00:00.000Z,Restaurant,-78.847625,35.662219
2,4092018021,Eggs Up Grill #31,HOLLY SPRINGS,2019-01-03T00:00:00.000Z,Restaurant,-78.794407,35.664360
4,4092016977,MCDONALDS #32242,HOLLY SPRINGS,2015-07-14T00:00:00.000Z,Restaurant,-78.831080,35.636251
5,4092017192,STARBUCKS #19453,HOLLY SPRINGS,2016-04-05T00:00:00.000Z,Restaurant,-78.849481,35.662665
7,4092017828,Panera Bread #6039,HOLLY SPRINGS,2018-04-10T00:00:00.000Z,Restaurant,-78.844724,35.665065
8,4092015574,THAI THAI CUISINE,HOLLY SPRINGS,2010-02-16T00:00:00.000Z,Restaurant,-78.832242,35.677791
9,4092017643,Texas Roadhouse #594,HOLLY SPRINGS,2017-07-13T00:00:00.000Z,Restaurant,-78.846159,35.663754
11,4092017679,Firehouse Subs #1052,HOLLY SPRINGS,2017-09-18T00:00:00.000Z,Restaurant,-78.837741,35.640817
13,4092016842,DQ GRILL &amp; CHILL #43767,HOLLY SPRINGS,2014-12-29T00:00:00.000Z,Restaurant,-78.835929,35.639829


In [18]:
wcres['RESTAURANTOPENDATE']=pd.to_datetime(wcres['RESTAURANTOPENDATE'])
wcres['RESTAURANTOPENDATE']=wcres['RESTAURANTOPENDATE'].dt.date
wcres['RESTAURANTOPENDATE']
wcres.to_csv('wcres.csv')
wcres.shape

c:\users\itdxv01\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\itdxv01\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(2364, 7)

In [19]:
wcres.head(5)

,HSISID,NAME,CITY,RESTAURANTOPENDATE,FACILITYTYPE,X,Y
0,4092017831,DUNKIN DONUTS #350292,HOLLY SPRINGS,2018-04-25,Restaurant,-78.830142,35.637386
1,4092017308,WENDY'S #6319,HOLLY SPRINGS,2016-09-02,Restaurant,-78.847625,35.662219
2,4092018021,Eggs Up Grill #31,HOLLY SPRINGS,2019-01-03,Restaurant,-78.794407,35.664360
4,4092016977,MCDONALDS #32242,HOLLY SPRINGS,2015-07-14,Restaurant,-78.831080,35.636251
5,4092017192,STARBUCKS #19453,HOLLY SPRINGS,2016-04-05,Restaurant,-78.849481,35.662665


In [20]:
import reverse_geocoder as rg

#for NAME,CITY,Y,X in zip(wcres['NAME'],wcres['CITY'],wcres['Y'],wcres['X']):       
#     if CITY is np.nan and X!=0 and Y!=0:        
#         coordinates =(Y,X)       
#         result = rg.search(coordinates)
#         for key,value in result[0].items(): 
#             if key=='name':                
#                 print(NAME,CITY,value)                

for index,row in wcres.iterrows():
    if row['CITY'] is np.nan and row['X']!=0 and row['Y']!=0:        
        coordinates =(row['Y'],row['X'])       
        result = rg.search(coordinates)
        for key,value in result[0].items(): 
            if key=='name':
                wcres['CITY'][index]=value.upper()

Loading formatted geocoded file...


c:\users\itdxv01\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\itdxv01\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
wcres.to_csv('wcres_Upper.csv')
wcres_upper=pd.read_csv('wcres_Upper.csv')
wcres_upper.shape

(2364, 8)

In [32]:
wcres=pd.read_csv('wcres_upper.csv')
wcres = wcres[pd.notnull(wcres['CITY'])]
wcres.to_csv('wcres_Upper_notnull.csv')
wcres.shape

(2272, 8)

In [30]:
ss=pd.merge(newarr,wcres,on='HSISID')
ss.shape
ss.to_csv('ss.csv')

In [31]:
wakerestlist=pd.read_csv('wakecounty_restaurants_withoutmissingvalues.csv')
wakerestlist.columns

Index(['Unnamed: 0', 'HSISID', 'NAME', 'CITY', 'RESTAURANTOPENDATE',
       'FACILITYTYPE', 'X', 'Y'],
      dtype='object')

In [44]:
ss1=pd.merge(wakerestlist,newarr,on='HSISID')
ss1.shape
ss1.columns
ss3=ss1.drop(columns=['DATE_', 'DESCRIPTION',
       'TYPE', 'INSPECTOR', 'PERMITID','RESTAURANTOPENDATE'])
ss3.to_csv('ss3.csv')
ss1.columns

Index(['Unnamed: 0', 'HSISID', 'NAME', 'CITY', 'RESTAURANTOPENDATE',
       'FACILITYTYPE', 'X', 'Y', 'OBJECTID', 'SCORE', 'DATE_', 'DESCRIPTION',
       'TYPE', 'INSPECTOR', 'PERMITID'],
      dtype='object')

In [34]:
ss2=pd.read_csv('xy.csv')
ss2.columns

Index(['X', 'Y'], dtype='object')

In [35]:
wakerestlist['X']=wakerestlist['X'].map(ss2.set_index('X')['X'])

KeyError: 'X'

In [22]:
restaurantsviolations=pd.read_csv("C:\Python3\Coursera_Captstone\Food_Inspection_Violations.csv") 

c:\users\itdxv01\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
restaurantsviolations.shape

In [ ]:
wcres.sort_values(by='CITY')
wcres.columns

In [ ]:
newarr.rename(columns={'DATE_':'InspectionDate','SCORE':'SanitaryScores','X':'Long','Y':'Lat'},inplace=True)
foodinspec=newarr.drop(['OBJECTID','INSPECTOR','PERMITID','DESCRIPTION','TYPE'],axis=1)
foodinspec.columns

In [ ]:
wcrir=pd.merge(wcres,foodinspec,on=['HSISID'],how='inner')

In [ ]:
wcrir.sort_values(by='CITY')
wcrir.to_csv('wcrir.csv')
wcrir.columns
wcrir.shape

In [ ]:
sanitaryrating=wcrir.to_csv('sanitary.csv')

In [37]:
# CLIENT_ID = 'EA0WZQZP3HJVT4DTV02NLEGNIFSAHPTRXK50TAZIIYY5A5OG' # your Foursquare ID
# CLIENT_SECRET = '5X1OY3KPXNEM1F5CVU2A124DQUNH3GC0TDP1MP1M0FX31D1Z' # your Foursquare Secret

CLIENT_ID ='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
CLIENT_SECRET ='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'
VERSION = '20190104'
LIMIT = 5000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=5000
latitude = 35.507104
longitude = -78.739181
search_query='Restaurants'


Your credentails:
CLIENT_ID: BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB
CLIENT_SECRET:WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW


In [ ]:
restID=[]
restName=[]
restCity=[]
restLat=[]
restLng=[]
restRate=[]

In [ ]:
for lat ,lng in zip(citylatlng['Latitude'],citylatlng['Longitude']):  
    url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng, VERSION,search_query, radius, LIMIT)
    results = requests.get(url1).json()
    i=results['response']['groups'][0]['items']
#    print(i)
#     i[0]['venue']['id']
#     for res in range(0,len(i)):
#         if 'city' in i[res]['venue']['location'].keys():
#             print(i[res]['venue']['id'],i[res]['venue']['name'],i[res]['venue']['location']['city'])
#     url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(i[res]['venue']['id'], CLIENT_ID, CLIENT_SECRET, VERSION)        
#     result2 = requests.get(url2).json() 
#     print(result2)
#    print(i[res]['venue'].keys())
#     if 'rating' in result2['response']['venue'].keys():
#         restID.append(i[res]['venue']['id'])
#         restName.append(i[res]['venue']['name'])
#         restCity.append(i[res]['venue']['location']['city'])        
#         restLat.append(i[res]['venue']['location']['lat'])
#         restLng.append(i[res]['venue']['location']['lng'])
#         restRate.append(result2['response']['venue']['rating'])
#     else:
#         restID.append(i[res]['venue']['id'])
#         restName.append(i[res]['venue']['name'])
#         restCity.append(i[res]['venue']['location']['city'])   
#         restLat.append(i[res]['venue']['location']['lat'])
#         restLng.append(i[res]['venue']['location']['lng'])
#         restRate.append('NA')

# url2='https://api.foursquare.com/v2/venues/4c50b335250dd13a88d3fe7b?client_id=EA0WZQZP3HJVT4DTV02NLEGNIFSAHPTRXK50TAZIIYY5A5OG&client_secret=5X1OY3KPXNEM1F5CVU2A124DQUNH3GC0TDP1MP1M0FX31D1Z&v=20190104'
# result2 = requests.get(url2).json()
# print(result2)


In [ ]:
# CLIENT_ID = '3FOP3IJUAPXF2DQX22YBSYMIIQOHBH0QOOR5NVIMLWAXBN4K' # your Foursquare ID /dEVNATH
# CLIENT_SECRET = '4C5QPIRQ5OSQZD1BZE0N2IZMR5QU2JUOMQ1BZHCJ3RHSISSN' # your Foursquare Secret
# restID=[]
# restName=[]
# restCat=[]
# restCity=[]
# restPost=[]
# restLat=[]
# restLng=[]
# restRate=[]
# for city,lat ,lng in zip(citylatlng['City'],citylatlng['Latitude'],citylatlng['Longitude']): 
#     url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng, VERSION,search_query, radius, LIMIT)    
#     results = requests.get(url1).json()
#     r=results['response']['groups'][0]['items']    
#     for i in range(0,len(r)):
# #        print(city,r[i]['venue']['name'],r[i]['venue']['categories'])
#             print(i,city,r[i]['venue']['id'],r[i]['venue']['name'],r[i]['venue']['categories'][0]['name'])#,r[i]['venue']['categories'][0]['name'])#,r[i]['venue']['location']['postalCode'],r[i]['venue']['location']['lat'],r[i]['venue']['location']['lng'])
#             restID.append(r[i]['venue']['id'])
#             restName.append(r[i]['venue']['name'])
#             restCat.append(r[i]['venue']['categories'][0]['name'])            
#             if 'city' in r[i]['venue']['location']:            
#                 restCity.append(r[i]['venue']['location']['city'])
#         except:
#             pass
#fourSquareData=pd.DataFrame(list(zip(restID,restName,restCat,restCity)),columns=['ID','Name','Category','City'])
#fourSquareData
#         else:
#             restCity.append('NA')
#         if 'postalCode' in  r[i]['venue']['location']:
#             restPost.append(r[i]['venue']['location']['postalCode'])
#         else:
#             restPost.append('NA')    
#         restLat.append(r[i]['venue']['location']['lat'])
#         restLng.append(r[i]['venue']['location']['lng'])
#results['meta']
#i=results['response']['groups'][0]['items']
# r=results['response']['groups'][0]['items']
# print(len(r))
# r[4]['venue']['name']
#r[4]['venue']['id']
#r[1]['venue']['location']['postalCode']
#r[4]['venue']['categories'][0]['name']
# dataframe=json_normalize(len(r))
# dataframe
#fourSquareData=pd.DataFrame(list(zip(restID,restName,restCat,restCity,restPost,restLat,restLng)),columns=['ID','Name','Category','City','Zipcode','Lat','Lng'])
#fourSquareData

In [ ]:
#             print(i,city,r[i]['venue']['id'],r[i]['venue']['name'],r[i]['venue']['categories'][0]['name'])#,r[i]['venue']['categories'][0]['name'])#,r[i]['venue']['location']['postalCode'],r[i]['venue']['location']['lat'],r[i]['venue']['location']['lng'])
#             restID.append(r[i]['venue']['id'])
#             restName.append(r[i]['venue']['name'])
#             restCat.append(r[i]['venue']['categories'][0]['name'])            
#             if 'city' in r[i]['venue']['location']:            
#                 restCity.append(r[i]['venue']['location']['city'])
#         except:
#             pass
#fourSquareData=pd.DataFrame(list(zip(restID,restName,restCat,restCity)),columns=['ID','Name','Category','City'])
#fourSquareData
#         else:
#             restCity.append('NA')
#         if 'postalCode' in  r[i]['venue']['location']:
#             restPost.append(r[i]['venue']['location']['postalCode'])
#         else:
#             restPost.append('NA')    
#         restLat.append(r[i]['venue']['location']['lat'])
#         restLng.append(r[i]['venue']['location']['lng'])
#results['meta']
#i=results['response']['groups'][0]['items']
# r=results['response']['groups'][0]['items']
# print(len(r))
# r[4]['venue']['name']
#r[4]['venue']['id']
#r[1]['venue']['location']['postalCode']
#r[4]['venue']['categories'][0]['name']
# dataframe=json_normalize(len(r))
# dataframe
#fourSquareData=pd.DataFrame(list(zip(restID,restName,restCat,restCity,restPost,restLat,restLng)),columns=['ID','Name','Category','City','Zipcode','Lat','Lng'])
#fourSquareData

In [ ]:
fourSquareData.shape

In [ ]:
fourSquareData

In [ ]:
City=[]
for id in fourSquareData['ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(id, CLIENT_ID, CLIENT_SECRET, VERSION)
    results = requests.get(url).json()
    print(results)
    if 'lat' in results['response']['venue']['location']:        
        restLat.append(results['response']['venue']['location']['lat'])
    else:
        restLat.append('NA')
    if 'lng' in results['response']['venue']['location']:        
        restLng.append(results['response']['venue']['location']['lng'])
    else:
        restLng.append('NA')
    if 'rating' in results['response']['venue']:
        restRate.append(results['response']['venue']['rating'])
    else:
        restRate.append('NA')        
    if 'city' in results['response']['venue']['location']:        
        City.append(results['response']['venue']['location']['city'])
    else:
        City.append('NA')
    RatingsData=pd.DataFrame(list(zip(restLat,restLng,restRate,restCity)),columns=['Latitude','Longitude','Rating','City'])    
#    restRate.append(results['response']['venue']['rating'])
#     restLat.append(results['venue']['location']['lat'])
#     restLng.append(results['venue']['location']['lng'])
#     if 'rating' in results['response']['venue']:
#         restRate.append(i[res]['response']['venue']['rating'])
#     else:
#         restRate.append('Not rated')
RatingsData.head(5)

In [ ]:
fourSquareData.head(5)

In [ ]:
RatingsData.shape

In [ ]:
fourSquareData.shape

In [ ]:
RatingsData.to_csv('FourSquareRatings.csv')

In [ ]:
fourSquareData.to_csv('FourSquareData.csv')

In [ ]:
rd=pd.merge()

In [ ]:
# CLIENT_ID = '3FOP3IJUAPXF2DQX22YBSYMIIQOHBH0QOOR5NVIMLWAXBN4K' # your Foursquare ID /dEVNATH
# CLIENT_SECRET = '4C5QPIRQ5OSQZD1BZE0N2IZMR5QU2JUOMQ1BZHCJ3RHSISSN' # your Foursquare Secret

# CLIENT_ID = 'WIAJSGOJPMXIL4TLISGU1DF2SIR0C0ZWJGPMBSRTFDXBBJPL' # your Foursquare ID
# CLIENT_SECRET = 'KRIYBP50TYU5G235JTDTOVRHD4QQKOSD52VYBKUFL2USKRYR' # your Foursquare Secret

# CLIENT_ID='EA0WZQZP3HJVT4DTV02NLEGNIFSAHPTRXK50TAZIIYY5A5OG'
# CLIENT_SECRET='5X1OY3KPXNEM1F5CVU2A124DQUNH3GC0TDP1MP1M0FX31D1Z'


# CLIENT_ID='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
# CLIENT_SECRET='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'



VERSION = '20190104'
LIMIT = 5000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=5000
latitude = 35.507104
longitude = -78.739181
search_query='Restaurants'

In [ ]:
restID=[]
restName=[]
restCat=[]
restCity=[]
restPost=[]
restLat=[]
restLng=[]
restRate=[]
for city,lat ,lng in zip(citylatlng['City'],citylatlng['Latitude'],citylatlng['Longitude']): 
    url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng, VERSION,search_query, radius, LIMIT)    
    results = requests.get(url1).json()
    r=results['response']['groups'][0]['items']
    for i in range(0,len(r)):
        print(len(r[i]['venue']['categories']))
        if (len(r[i]['venue']['categories']))==1:            
            restID.append(r[i]['venue']['id'])
            restName.append(r[i]['venue']['name'])
            restCat.append(r[i]['venue']['categories'][0]['name'])
            restCity.append(city)
#             if 'city' in r[i]['venue']['location']:            
#                 restCity.append(r[i]['venue']['location']['city']
# fourSquareData=pd.DataFrame(list(zip(restID,restName,restCat,restCity)),columns=['ID','Name','Category','City'])

In [ ]:
fourSquareData=pd.DataFrame(list(zip(restID,restName,restCat,restCity)),columns=['ID','Name','Category','City'])
fourSquareData.shape

In [ ]:
fourSquareData.head(1000)

In [ ]:
fourSquareData.to_csv('FQ_New.csv')

In [ ]:
# id=fourSquareData['ID']
# fourSquareData['ID'][0]
len(fourSquareData['ID'])

In [34]:
hsisid=pd.read_csv('hsisid_xy_00.csv')
hsisid
restID=[]
City=[]
# CLIENT_ID='EA0WZQZP3HJVT4DTV02NLEGNIFSAHPTRXK50TAZIIYY5A5OG'
# CLIENT_SECRET='5X1OY3KPXNEM1F5CVU2A124DQUNH3GC0TDP1MP1M0FX31D1Z'
CLIENT_ID='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
CLIENT_SECRET='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'
import time
for i in range(0,len(fourSquareData['ID'])):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(fourSquareData['ID'][i], CLIENT_ID, CLIENT_SECRET, VERSION)
    print(i,fourSquareData['ID'][i],url)
    results = requests.get(url).json()
    restID.append(fourSquareData['ID'][i])
    if 'lat' in results['response']['venue']['location']:        
        restLat.append(results['response']['venue']['location']['lat'])
    else:
        restLat.append('NA')
    if 'lng' in results['response']['venue']['location']:        
        restLng.append(results['response']['venue']['location']['lng'])
    else:
        restLng.append('NA')
    if 'rating' in results['response']['venue']:
        restRate.append(results['response']['venue']['rating'])
    else:
        restRate.append('NA')        
    if 'city' in results['response']['venue']['location']:        
        City.append(results['response']['venue']['location']['city'])
    else:
        City.append('NA')
RD2=pd.DataFrame(list(zip(restID,restLat,restLng,restRate,City)),columns=['ID','Latitude','Longitude','Rating','City'])    
#    restRate.append(results['response']['venue']['rating'])
#     restLat.append(results['venue']['location']['lat'])
#     restLng.append(results['venue']['location']['lng'])
#     if 'rating' in results['response']['venue']:
#         restRate.append(i[res]['response']['venue']['rating'])
#     else:
#         restRate.append('Not rated')


,venueid
0,4092015755
1,4092012825
2,4092013979
3,4092021901
4,4092017584
5,4092020996
6,4092011804
7,4092010072
8,4092016509
9,4092016069


In [ ]:
RD2.to_csv('RD2.csv')

In [ ]:
RD1.to_csv('RD1.csv')

In [ ]:
RatingsData.to_csv('RatingsData_003.csv')

In [ ]:
RatingsData

In [ ]:
RatingsData.to_csv('RatingsData_004.csv')

In [ ]:
RatingsData=[]
City=[]
restID=[]
restName=[]
restCat=[]
restCity=[]
restPost=[]
restLat=[]
restLng=[]
restRate=[]
CLIENT_ID='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
CLIENT_SECRET='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'
import time
for id in venues:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(id, CLIENT_ID, CLIENT_SECRET, VERSION)
    results = requests.get(url).json()
    restID.append(id)
    print(id)
    try:
        
        if 'lat' in results['response']['venue']['location']:        
            restLat.append(results['response']['venue']['location']['lat'])
        else:
            restLat.append('NA')        
        if 'lng' in results['response']['venue']['location']:        
            restLng.append(results['response']['venue']['location']['lng'])
        else:
            restLng.append('NA')
        if 'rating' in results['response']['venue']:
            restRate.append(results['response']['venue']['rating'])
        else:
            restRate.append('NA')        
        if 'city' in results['response']['venue']['location']:        
            City.append(results['response']['venue']['location']['city'])
        else:
            City.append('NA')
    except:
        pass
RatingsData=pd.DataFrame(list(zip(restID,restLat,restLng,restRate,City)),columns=['ID','Latitude','Longitude','Rating','City'])    

In [ ]:
RestaurantRatingsData=pd.read_csv('RatingsData_001.csv')

In [ ]:
RestaurantRatingsData.columns

In [ ]:
RestaurantRatingsData.drop(columns=['Unnamed: 0'])

In [ ]:
RestaurantRatingsData.shape

In [ ]:
RestaurantRatingsData

In [ ]:
fourSquareData

In [ ]:
fourSquareData
#fourSquareData.to_csv('RatingsData_003.csv')

In [ ]:
RestData=pd.read_csv('RD.csv')

In [ ]:
RestData.shape

In [ ]:
RestData.drop(columns=['Unnamed: 0'])

In [ ]:
fourSquareData.shape

In [ ]:
fourSquareData

In [ ]:
fqrestdata=pd.merge(fourSquareData,RestData,left_index=True, right_index=True)

In [ ]:
fqrestdata.to_csv('fqrestdata.csv')

In [ ]:
fqrestdata

In [ ]:
fqrestdata.drop(columns=['ID_y','City_y','Unnamed: 0'])

In [ ]:
fqrestdata.rename(columns={'ID_x':'VenueID','City_x':'City'},inplace=True)
newfqrestdata=fqrestdata.drop(columns=['ID_y','City_y','Unnamed: 0'])

In [ ]:
newfqrestdata.to_csv('RestaurantFQ_Data.csv')

In [ ]:
newfqrestdata=pd.read_csv('RestaurantFQ_Data.csv')

In [ ]:
newfqrestdata['Category'].value_counts()

In [ ]:
cuisineratingdata=pd.read_csv('FQ_Cusine.csv')


In [ ]:
cuisineratingdata.drop(columns=['Unnamed: 0'])
cuisineratingNA=cuisineratingdata.dropna()

In [ ]:
cna=cuisineratingNA.drop(columns=['Unnamed: 0'])
cna['Cuisine'].value_counts()


In [ ]:
cna.groupby(['City','Cuisine']).agg([np.average])

In [ ]:
cna
cna.shape

In [ ]:
newarr.shape

In [ ]:
m = folium.Map(location=[35.7803977, -78.6390989], zoom_start=12,tiles='Stamen Terrain')


# folium.Marker(
#     location=[35.743223,-78.822297,],
#     popup='Restaurant 1'
# ).add_to(m)

# folium.Marker(
#     location=[35.750153	, -78.877270],
#     popup='Restaurant 2'
# ).add_to(m)

# folium.Marker(
#     location=[35.717046, -78.840340],
#     popup='Restaurant 3'
# ).add_to(m)

for lat,lng, rating,Name in zip(cna['Latitude'],cna['Longitude'],cna['Rating'],cna['Name']):
    folium.Marker(
    location=[lat,lng],
    popup=Name
    
).add_to(m)
m

In [ ]:
cna.shape


In [ ]:
ccc=newarr
ccc.columns

In [ ]:
m = folium.Map(location=[35.7803977, -78.6390989], zoom_start=12,tiles='Stamen Terrain')


# folium.Marker(
#     location=[35.743223,-78.822297,],
#     popup='Restaurant 1'
# ).add_to(m)

# folium.Marker(
#     location=[35.750153	, -78.877270],
#     popup='Restaurant 2'
# ).add_to(m)

# folium.Marker(
#     location=[35.717046, -78.840340],
#     popup='Restaurant 3'
# ).add_to(m)

for lat,lng, rating,Name in zip(cna['Latitude'],cna['Longitude'],cna['Rating'],cna['Name']):
    folium.Marker(
    location=[lat,lng],
    popup=Name
    
).add_to(m)
m

In [ ]:
fiviolation=pd.read_csv("C:\Python3\Coursera_Captstone\Food_Inspection_Violations.csv")

In [ ]:
fiviolation.shape

In [ ]:
fvio=pd.DataFrame()
fvio=fiviolation.groupby(['HSISID']).count().sort_index(by='OBJECTID')

In [ ]:
fvio

In [ ]:
mergedDF=newarr.merge(fvio,how='inner',on=['HSISID'])

In [ ]:
mergedDF.head(100)

In [ ]:
mergedDF.to_csv('mergedDF.csv')

In [ ]:
emr=mergedDF.merge(wcres,how='inner',on=['HSISID'])

In [ ]:
emr.to_csv('emr.csv')

In [ ]:
emr.shape

In [ ]:
emr

In [ ]:
TipsData=[]
venueid=[]
tips=[]
CLIENT_ID='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
CLIENT_SECRET='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'
limit=5
import time
# for id in zip(RestData['ID']):
for id in range(0,100):    
    print(RestData['ID'][id])
    url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(RestData['ID'][id],CLIENT_ID, CLIENT_SECRET, VERSION, limit)
    print(results)
#     results = requests.get(url).json()    
#     try:
#         tips=result['response']['venue']['tips']['count']
        
#     except:
#         pass
# TipsData=pd.DataFrame(list(zip(id,tips)),columns=['ID','TIPS'])    


In [ ]:
TipsData

In [ ]:
parks=pd.read_csv('Parks_in_Wake_County.csv')
parks.head()

In [ ]:
m = folium.Map(location=[35.7803977, -78.6390989], zoom_start=12,tiles='Mapbox Bright')


# folium.Marker(
#     location=[35.743223,-78.822297,],
#     popup='Restaurant 1'
# ).add_to(m)

# folium.Marker(
#     location=[35.750153	, -78.877270],
#     popup='Restaurant 2'
# ).add_to(m)

# folium.Marker(
#     location=[35.717046, -78.840340],
#     popup='Restaurant 3'
# ).add_to(m)

for lat,lng, Name in zip(parks['Y'],parks['X'],parks['NAME']):
    folium.Marker(
    location=[lat,lng],
    popup=Name
    
).add_to(m)
m

In [ ]:
cna

In [ ]:
cus=pd.get_dummies(cna['Cuisine'])
restdata=ccna.join(cus)

In [ ]:
restdata.to_csv('restdata.csv')

In [71]:
restdata=pd.read_csv('restdata.csv')

In [ ]:
restdata['Category'].value_counts()

In [75]:
angierdata=pd.read_csv('angier_restdata.csv')

In [84]:
import geojson
with open('Angier_Zoning.geojson') as f:
    gj = geojson.load(f)
features = gj['features'][0]
gj

{"features": [{"geometry": {"coordinates": [[[[2075562.906386152, 637528.7917895019], [2075602.6562741995, 637525.0311494917], [2075613.5936181992, 637859.9167818129], [2075962.6667705327, 637857.4479178041], [2076279.7811388224, 637842.8546378016], [2076366.9481149018, 637854.0111178011], [2076377.3749949187, 637907.0000458509], [2076427.8853309602, 637912.2293578535], [2076528.8020670563, 637925.6043338776], [2076630.4791231453, 637939.0737738907], [2076683.6667072028, 637946.3541578948], [2076833.5938753337, 637966.5313099176], [2076862.843667373, 637974.1979979128], [2076863.7291713655, 637970.3229259104], [2076885.520915389, 637874.9270858318], [2076919.9480514228, 637879.6144458354], [2076924.895763427, 637853.604845807], [2076937.614611432, 637779.906285733], [2076941.980435446, 637780.5447497368], [2076978.8431554735, 637610.1360455751], [2077109.8526915908, 637638.4752456099], [2077122.3070916086, 637641.1683656126], [2077124.7605956048, 637629.9276615977], [2077135.6979396194

In [85]:
m = folium.Map(location=[35.5071045, -78.7391813], zoom_start=11,)
 
# Add the color for the chloropleth:
m.choropleth(
 geo_data=gj,
 name='choropleth',
 data=angierdata,
 columns=['Name', 'Rating'],
 key_on='feature.Rating',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Unemployment Rate (%)'
)
folium.LayerControl().add_to(m)
m


In [ ]:
Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5
​
for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')